<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/ibd_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Screening Using IBD's RS Rating Methodology

This notebook demonstrates stock screening techniques inspired by the methodology used in Investor's Business Daily (IBD). It incorporates the (price) RS Rating, EPS RS Rating, and Revenue RS Rating to evaluate stocks based on price performance, earnings, and revenue growth. The Price RS Rating typically reflects a stock’s performance over the past year, but a 3-month version is also available to assist with short-term trading decisions.

## Install and Setup (this section will be executed automatically)

### Install Required Packages

In [ ]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ranking_utils"
%pip install requests-cache

!wget -O NotoSansTC-Regular.ttf https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ranking_utils) to /tmp/pip-req-build-cgpczyhz
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-cgpczyhz
  Running command git checkout -b feature/ranking_utils --track origin/feature/ranking_utils
  Switched to a new branch 'feature/ranking_utils'
  Branch 'feature/ranking_utils' set up to track remote branch 'feature/ranking_utils' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit 2aed48fc5c43e0b3761fa04e0b9daebef7207dee
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.5 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.7.0-py3-none-any.whl size=89084 sha256=ce5cd1fcce4084876cd4001e2e582375e005bb78e55431b64ae60385d8c62a83
  Stored in directory: /tmp/pip-ephem-wheel-cache-3x87uvq5/wheels/37/13/2b/a599ed1c9af840bb5c5d20b1aa8e13e50523bc0d554a13952d
Successfully bu

### Setup and Configuration

In [ ]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [ ]:
# @title Set Chinese Font for matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fm.fontManager.addfont('NotoSansTC-Regular.ttf')
font_name = 'Noto Sans TC'
if font_name not in plt.rcParams['font.sans-serif']:
    plt.rcParams['font.sans-serif'].insert(0, font_name)

In [ ]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'
        self.branch = branch

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        if dir_path:
            url = f'{self.api_url}/{dir_path}?ref={self.branch}'
        else:
            url = f'{self.api_url}?ref={self.branch}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        elif response.status_code == 404:
            print(f"Directory not found: {url}")
            return []
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='yorkjong',
    repo_name='stock-reports',
    dir='ibd',
    branch='data'
)

In [ ]:
# @title Taiwan Stock Name Lookup

class StockNameLookup:
    _df = None  # Class-level variable to hold the DataFrame

    @classmethod
    def _load_data(cls):
        if cls._df is None:  # Check if the DataFrame is already loaded
            gh = ReadOnlyGitHub(
                repo_owner='yorkjong',
                repo_name='stock-reports',
                dir='data/stock_list',
            )
            cls._df = gh.download_csv('taiwan_stock_OpenAPI.csv')

    @classmethod
    def tw_stock_name(cls, ticker):
        cls._load_data()  # Ensure data is loaded before accessing

        code = ticker.split('.')[0]  # Extract the code part

        # Filter the DataFrame to find the stock name for the given code
        stock_name = cls._df.loc[cls._df['Code'] == code, 'Name']

        # Check if the stock_name is empty and return an appropriate message
        if not stock_name.empty:
            return stock_name.values[0]  # Return the first matched stock name
        else:
            return None  # Return None if ticker not found


def tw_stock_name(ticker):
    return StockNameLookup.tw_stock_name(ticker)

In [ ]:
# @title Metadata

def parse_metadata(filename):
    components = filename.split('_')
    if len(components) != 5:
        raise ValueError("Filename does not have the expected number of components")

    source, kind, period, rs_win, date = components
    rs_win = rs_win.replace('ibd', '')
    date = date.replace('.csv', '')

    return {
        "Source": source,
        "Type": kind,
        "Period": period,
        "RS window": rs_win,
        "Date": date
    }


def print_metadata(meatadata):
    for key, value in meatadata.items():
        print(f"{key}: {value}")

In [ ]:
# @title IBD Financial files
import re
from datetime import datetime


def get_latest_file(file_list, source):
    pattern = rf'^{re.escape(source)}_(\w+)_fin_(\d+)\.csv$'
    matching_files = []

    for file in file_list:
        match = re.match(pattern, file)
        if match:
            date_str = match.group(2)
            date = datetime.strptime(date_str, "%Y%m%d")
            matching_files.append((file, date))

    if not matching_files:
        return None

    return max(matching_files, key=lambda x: x[1])[0]


def fin_download_latest_csv(source):
    gh = ReadOnlyGitHub(
        repo_owner='yorkjong',
        repo_name='stock-reports',
        dir='ibd_fin',
        branch='data'
    )
    file_list = gh.list_filenames()
    fin_filename = get_latest_file(file_list, source)
    df = gh.download_csv(fin_filename)
    return df, fin_filename

In [ ]:
# @title DataFrame Utilities

def print_column(df, column):
    if column in df.columns:
        print(', '.join(df[column]))

In [ ]:
# @title Source of Tickers

def tickers_from_df(df):
    if 'Name' in df.columns:
        return [name.strip() for names in df['Name']
                for name in names.split(',')]
    elif 'Ticker' in df.columns:
        return [ticker.strip() for tickers in df['Ticker']
                for ticker in tickers.split(',')]
    return []

def major_indices():
    return ['^DJI', '^IXIC', '^NDX', '^RUT', '^SOX',
            '^TWII', '^N225', '^HSI',
            '^STOXX50E', '^FTSE', '^GDAXI', '^FCHI', '^GSPTSE']

def sector_indices():
    return ['SOXX', 'DVY',
            'IWB','IWM', 'IWV',  'IJR',
            'ITB', 'IHI', 'IYC', 'ITA', 'IAK',
            'IYZ', 'IYT', 'IYR', 'IYF', 'IYJ',
            'IYG', 'IYH', 'IYK', 'IDU', 'IYE', 'IHE',
            'IAT', 'IAI', 'IEO', 'IYM', 'IHF']

In [ ]:
# @title Checkboxes

import ipywidgets as widgets

def cbs_create(symbols, n_pre_checked=10):
    '''Create a list of checkboxes'''
    return [
        widgets.Checkbox(
            value=(i < n_pre_checked),  # Set first n items as checked
            description=symbol,
            layout=widgets.Layout(width='auto'),
            style={'description_width': 'auto'}
        )
        for i, symbol in enumerate(symbols)
    ]

def cbs_with_grid(checkboxes, n_cols=5):
    '''Create a grid layouting the given checkboxes'''
    return widgets.GridBox(checkboxes, layout=widgets.Layout(
        width='auto',
        grid_template_columns=f'repeat({n_cols}, 1fr)',
        grid_gap='10px'  # Add some space between the checkboxes
    ))

def cbs_get_selected(checkboxes):
    '''Get the selected symbols from the given checkboxes'''
    return [checkbox.description for checkbox in checkboxes if checkbox.value]

def cbs_unselect_all(checkboxes):
    '''Unselect all checkboxes in the given list'''
    for checkbox in checkboxes:
        checkbox.value = False

def cbs_select_top(checkboxes, n=10):
    '''Select the top n checkboxes in the given list'''
    for i, checkbox in enumerate(checkboxes):
        checkbox.value = (i < n)

In [ ]:
# @title Dropdown Menus

def create_period_dropdown(value='2y'):
    return widgets.Dropdown(
        options=['1y', '2y', '5y'],
        value=value,
        description='Period:',
    )

def create_interval_dropdown(value='1wk'):
    return widgets.Dropdown(
        options=['1d', '1wk'],
        value=value,
        description='Interval:',
    )

def create_rs_window_dropdown(value='3mo'):
    return widgets.Dropdown(
        options=['3mo', '12mo'],
        value=value,
        description='rs_window:',
    )

def create_style_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['default', 'classic', 'yahoo', 'charles', 'tradingview', 'binance', 'binancedark', 'mike', 'nightclouds', 'checkers', 'ibd', 'sas', 'starsandstripes', 'kenan', 'blueskies', 'brasil'],
        value='yahoo' if value is None else value,
        description='Style:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

def create_template_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['plotly', 'plotly_white', 'plotly_dark', 'ggplot2', 'seaborn', 'simple_white', 'presentation', 'xgridoff', 'ygridoff'],
        value='plotly_dark' if value is None else value,
        description='Template:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

In [ ]:
# @title Multiple Searchable Dropdown Menus
import ipywidgets as widgets

def create_search_box():
    '''Create a Text widget for search input'''
    return widgets.Text(
        description='Search:',
        placeholder='Type to search',
        layout=widgets.Layout(width='auto')
    )

def create_dropdown(options, description='Stock:'):
    '''Create a Dropdown widget for displaying filtered options'''
    return widgets.Dropdown(
        description=description,
        options=[None] + options,  # None as the default option
        layout=widgets.Layout(width='auto'),
        value=None  # Set default value to None
    )

def update_dropdown(change, dropdown, options):
    '''Update the options in the dropdown based on search input'''
    search_text = change['new'].lower()
    filtered = [option for option in options if search_text in option.lower()]
    if filtered:
        dropdown.options = [None] + filtered
        dropdown.value = filtered[0]  # Auto-select the first matching option
    else:
        dropdown.options = [None]  # Retain only the None option if no match

def remove_duplicates_preserve_order(lst):
    '''Remove duplicates from a list while preserving order'''
    seen = set()
    result = []
    for item in lst:
        if item and item not in seen:
            seen.add(item)
            result.append(item)
    return result

def get_dropdowns_selected_options(dropdowns):
    '''Get selected options from the dropdowns'''
    selected = [dropdown.value for dropdown in dropdowns if dropdown.value]
    return remove_duplicates_preserve_order(selected)

def create_search_dropdowns(options, max_selections):
    '''Create a layout with search boxes and dropdowns'''
    # Create UI components
    search_boxes = [create_search_box() for _ in range(max_selections)]
    dropdowns = [create_dropdown(options) for _ in range(max_selections)]

    # Bind search box and dropdown menu events
    for search_box, dropdown in zip(search_boxes, dropdowns):
        search_box.observe(lambda change, dropdown=dropdown:
                            update_dropdown(change, dropdown, options),
                            names='value')
    # Create the layout
    controls = [widgets.HBox([search_box, dropdown])
                for search_box, dropdown  in zip(search_boxes, dropdowns)]
    layout = widgets.VBox(controls)
    return dropdowns, layout

In [ ]:
# @title Outputs
outputs = widgets.VBox()

In [ ]:
# @title enable_plotly_in_cell
# ref. https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

### Glossary of Terms

#### Source (The source of stocks to analyze)
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector:
  - For Taiwan Markets, possible values include:
    - **`TWSE`**: Taiwan Stock Exchange (台灣上市股票交易所)
    - **`TPEX`**: Taipei Exchange (上櫃交易所)
    - **`ESB`**: Emerging Stock Board (興櫃交易所)
  - Can also be combined with `+` (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`).
  - For America Markets, possible values include:
    - **`SPX`**: S&P 500 (標普五百指數)
    - **`DJIA`**: Dow Jones Industrial Average (道瓊指數)
    - **`NDX`**: NASDAQ-100 (納斯達克一百指數)
    - **`SOX`**: PHLX Semiconductor Index (費半指數)
  - Multiple indices can be combined using `+` (e.g., `SPX+DJIA+NDX+SOX`).

#### Period (Historical Data Time Range)
- The time range for which to fetch historical data.
  - **`2y`**: 2 years
  - **`6mo`**: 6 months

#### RS Window (Window for RS Calculation)
- The time window ('3mo' or '12mo') for Relative Strength:
  - **`3mo`**: 3 months
  - **`12mo`**: 12 months

#### RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
  - A value of 100 represents the performance of the benchmark index or market.
- The IBD RS calculates the performance over the last year, with the most recent quarter weighted double.
- The IBD 3-month RS calculates the performance over the last quarter.

#### RS Rating
- RS Rating, ranging from 1 (worst) to 99 (best), evaluates the price performance of a stock relative to a benchmark index.

# Execute Actions Step by Step


## Step 1. Select and Preview a File

---



In [86]:
# @title Step 1.1 Select a File

import ipywidgets as widgets

with requests_cache.disabled():
    filenames = github.list_filenames()

# Extract and sort all unique values in reverse order
all_dates = sorted(set(fn.split('_')[4].replace('.csv', '') for fn in filenames), reverse=True)
all_sources = sorted(set(fn.split('_')[0] for fn in filenames))
#all_types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)
#all_periods = sorted(set(fn.split('_')[2] for fn in filenames))
all_rs_windows = sorted(set(fn.split('_')[3].replace('ibd', '') for fn in filenames))

# Calculate the maximum length of filenames to set dropdown width
if filenames:
    max_filename_length = max(len(fn) for fn in filenames)
else:
    max_filename_length = 0
dropdown_width = f'{max_filename_length * 10}px'  # 10px width per character

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=all_dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=all_sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

rs_window_dropdown = widgets.Dropdown(
    options=all_rs_windows,
    description='RS window:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

output = widgets.Output()

def update_dropdowns(*args):
    # Filter files based on selected date
    date_filtered_files = [fn for fn in filenames if date_dropdown.value in fn]

    # Update Source dropdown
    available_sources = sorted(set(fn.split('_')[0] for fn in date_filtered_files))
    source_dropdown.options = available_sources
    if source_dropdown.value not in available_sources:
        source_dropdown.value = available_sources[0] if available_sources else None

    # Update RS window dropdown
    available_rs_windows = sorted(set(fn.split('_')[3].replace('ibd', '') for fn in date_filtered_files))
    rs_window_dropdown.options = available_rs_windows
    if rs_window_dropdown.value not in available_rs_windows:
        rs_window_dropdown.value = available_rs_windows[0] if available_rs_windows else None

    # Update file options
    update_file_options()

def update_file_options(*args):
    # Filter files based on selected date, source, and RS window
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value == fn.split('_')[0] and
            rs_window_dropdown.value == fn.split('_')[3].replace('ibd', ''))
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        global selected_file, metadata
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match
        selected_file = file_dropdown.value
        metadata = parse_metadata(selected_file)
        with output:
            output.clear_output()
            print_metadata(metadata)
    else:
        file_dropdown.value = None

# Bind event handlers
date_dropdown.observe(update_dropdowns, 'value')
source_dropdown.observe(update_file_options, 'value')
rs_window_dropdown.observe(update_file_options, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, rs_window_dropdown, file_dropdown, output)

# Initialize dropdowns
update_dropdowns()

Dropdown(description='Date:', layout=Layout(width='410px'), options=('20241013',), value='20241013')

Dropdown(description='Source:', layout=Layout(width='410px'), options=('TWSE+TPEX', 'U.S.Listed'), value='TWSE…

Dropdown(description='RS window:', layout=Layout(width='410px'), options=('12mo', '3mo'), value='12mo')

Dropdown(description='File:', layout=Layout(width='410px'), options=(), value=None)

Output()

In [93]:
# @title Step 1.2 Load and Preview the Files
print(f'{selected_file}:')
with requests_cache.disabled():
    df_rs = github.download_csv(selected_file)
display(df_rs)

with requests_cache.disabled():
    df_fin, fn_fin = fin_download_latest_csv(metadata['Source'])
print(f'\n{fn_fin}:')
display(df_fin)

print(f'\nMerged:')
df_rs2merge = df_rs.drop(['1wk:end max', 'Rating (3M:1M)', 'Rating (6M:3M)',
                          'Rating (9M:6M)'], axis=1)

columns_to_keep = ['Ticker', 'EPS RS', 'TTM EPS', 'Rev RS', 'TTM RPS', 'TTM PE']
df_fin2merge = df_fin[columns_to_keep]

df_merge = pd.merge(df_rs2merge, df_fin2merge, on='Ticker', how='left')
display(df_merge)

U.S.Listed_stocks_2y_ibd12mo_20241013.csv:


,Ticker,Sector,Industry,RS,1wk:end max,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Rating (3M:1M),Rating (6M:3M),Rating (9M:6M),Price,52W pos,MA50,MA200,Volume / VMA50
0,WGS,Healthcare,Health Information Services,747.67,768.64,709.43,794.56,663.41,311.46,99,99,99,97,60.00,1.00,37.91,20.96,1.70
1,FTEL,Consumer Cyclical,Specialty Retail,611.69,611.69,448.23,1173.25,868.23,344.33,99,99,99,98,16.88,0.42,15.97,12.36,0.64
2,SEZL,Financial Services,Credit Services,590.19,590.19,521.68,364.08,286.17,313.32,99,99,98,97,179.75,1.00,138.71,80.75,0.86
3,CHSN,Consumer Cyclical,Restaurants,554.48,683.75,683.75,100.23,152.80,116.62,99,51,90,73,12.24,0.80,3.29,2.19,0.00
4,ASTS,Technology,Communication Equipment,480.94,495.81,514.99,912.53,337.64,100.36,99,99,98,50,24.18,0.61,26.69,11.06,0.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,NDRA,Healthcare,Diagnostics & Research,1.92,2.04,1.89,15.72,26.07,80.57,1,1,1,14,0.21,0.00,0.91,23.21,0.17
5430,XPON,Industrials,Electrical Equipment & Parts,1.32,2.45,5.32,28.22,61.65,135.02,1,2,6,84,2.39,0.00,7.75,200.98,2.41
5431,CETX,Technology,Software—Infrastructure,1.15,5.81,25.81,22.99,56.93,66.66,1,2,5,7,0.44,0.00,7.71,101.57,16.16
5432,ADTX,Healthcare,Biotechnology,0.98,1.35,15.75,47.07,36.57,46.44,1,4,2,3,1.16,0.00,23.02,91.44,0.62



U.S.Listed_stocks_fin_20241013.csv:


,Ticker,Sector,Industry,Price,EPS QoQ (%),QoQ 2Q Algo (%),QoQ 3Q Algo (%),EPS YoY (%),YoY 2Q Algo (%),EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
0,LUNR,Industrials,Aerospace & Defense,7.2300,9.18,-7.84,-3.79,-4.58,NaN,6.003049e+09,-0.170,86.50,4.810,NaN,99.0,91.0
1,BZFD,Communication Services,Internet Content & Information,2.5100,3.84,-9.17,4.00,2.88,NaN,4.060600e+09,-1.120,-33.22,6.229,NaN,99.0,9.0
2,CART,Consumer Cyclical,Internet Retail,42.7400,-1.14,22.69,1.05,22000000.00,47000000.0,2.300000e+09,-5.560,10.62,13.986,NaN,99.0,76.0
3,ACDC,Energy,Oil & Gas Equipment & Services,7.3500,-37.89,55.43,-1.92,-19.82,NaN,1.882714e+09,-1.430,9.16,13.919,NaN,99.0,74.0
4,KNTK,Energy,Oil & Gas Midstream,48.5400,3.50,-13.17,7.10,0.32,NaN,1.644444e+09,2.830,22.00,24.316,17.56,99.0,83.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5052,ZENA,Technology,Software—Infrastructure,1.7700,0.41,0.29,-0.92,NaN,NaN,NaN,NaN,NaN,0.114,NaN,NaN,NaN
5053,ZIONO,Financial Services,Banks—Regional,26.7100,NaN,NaN,NaN,NaN,NaN,NaN,6.166,-8.37,20.043,4.32,NaN,35.0
5054,ZIONP,Financial Services,Banks—Regional,22.2300,NaN,NaN,NaN,NaN,NaN,NaN,6.166,-8.37,20.043,3.65,NaN,35.0
5055,ZK,Consumer Cyclical,Auto Manufacturers,24.5600,-0.17,-0.35,0.61,NaN,NaN,NaN,-4.940,NaN,314.047,NaN,NaN,NaN



Merged:


,Ticker,Sector,Industry,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Price,52W pos,MA50,MA200,Volume / VMA50,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE
0,WGS,Healthcare,Health Information Services,747.67,709.43,794.56,663.41,311.46,99,60.00,1.00,37.91,20.96,1.70,186.27,-4.38,2.538000e+01,9.334,NaN
1,FTEL,Consumer Cyclical,Specialty Retail,611.69,448.23,1173.25,868.23,344.33,99,16.88,0.42,15.97,12.36,0.64,NaN,NaN,NaN,NaN,NaN
2,SEZL,Financial Services,Credit Services,590.19,521.68,364.08,286.17,313.32,99,179.75,1.00,138.71,80.75,0.86,1035.74,7.05,4.857000e+01,33.982,25.5
3,CHSN,Consumer Cyclical,Restaurants,554.48,683.75,100.23,152.80,116.62,99,12.24,0.80,3.29,2.19,0.00,NaN,NaN,NaN,NaN,NaN
4,ASTS,Technology,Communication Equipment,480.94,514.99,912.53,337.64,100.36,99,24.18,0.61,26.69,11.06,0.39,-26.09,-1.30,-9.361111e+15,0.013,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,NDRA,Healthcare,Diagnostics & Research,1.92,1.89,15.72,26.07,80.57,1,0.21,0.00,0.91,23.21,0.17,274.47,-32.00,-8.510000e+00,NaN,NaN
5430,XPON,Industrials,Electrical Equipment & Parts,1.32,5.32,28.22,61.65,135.02,1,2.39,0.00,7.75,200.98,2.41,-65.97,-119.00,-6.690000e+00,70.928,NaN
5431,CETX,Technology,Software—Infrastructure,1.15,25.81,22.99,56.93,66.66,1,0.44,0.00,7.71,101.57,16.16,122.40,-170.40,-4.780000e+00,459.181,NaN
5432,ADTX,Healthcare,Biotechnology,0.98,15.75,47.07,36.57,46.44,1,1.16,0.00,23.02,91.44,0.62,542.87,-1375.20,1.167044e+14,12.114,NaN


## Step 2. Filter Stocks

In [94]:
# @title Filter 1. Sorting {"run":"auto"}
cond0 = "(df['RS'] > 100)" # @param ["(df['RS'] > 100)"]
cond1 = "& (df['Rating (RS)'] > 95)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond2 = "& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond3 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond4 = "& (df['Rev RS'] > 0)" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond5 = "& (df['TTM EPS'] > 0)" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond6 = "& (df['52W pos'] > 0.6)" # @param ["& (df['52W pos'] > 0.5)", "& (df['52W pos'] > 0.6)", "& (df['52W pos'] > 0.7)", ""]
cond7 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
sorted_column = "RS" # @param ["RS","1 Week Ago","1 Month Ago","3 Months Ago","6 Months Ago","Price","MA10","MA30","Volume / VMA10","EPS RS", "Rev RS"]
ascending = False # @param {"type":"boolean"}
num_items = 30 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 8
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df = df.sort_values(by=sorted_column, ascending=ascending)
df_top_f1 = df.head(num_items)
display(df_top_f1)

print_column(df_top_f1, 'Name')
print_column(df_top_f1, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 2y
RS window: 12mo
Date: 20241013


,Ticker,Sector,Industry,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Price,52W pos,MA50,MA200,Volume / VMA50,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE
2,SEZL,Financial Services,Credit Services,590.19,521.68,364.08,286.17,313.32,99,179.75,1.00,138.71,80.75,0.86,1035.74,7.05,4.857000e+01,33.982,25.50
24,DAVE,Technology,Software—Application,284.52,293.55,374.17,581.62,530.20,99,43.78,0.81,37.55,33.42,1.22,213.76,2.04,2.160000e+01,24.051,21.46
34,CVNA,Consumer Cyclical,Auto & Truck Dealerships,252.76,239.36,261.88,526.22,415.86,98,192.25,1.00,156.77,105.35,0.60,368.16,2.49,1.304000e+01,101.398,77.21
37,ADMA,Healthcare,Biotechnology,244.17,303.96,289.22,232.09,147.39,98,17.15,0.80,17.60,10.23,1.60,347.68,0.14,7.142000e+01,1.447,122.46
52,APP,Technology,Software—Application,217.35,213.65,163.52,236.00,238.26,98,147.00,1.00,104.21,76.67,0.90,231.25,2.34,3.302000e+01,11.726,62.82
53,GGAL,Financial Services,Banks—Regional,217.20,190.78,175.29,214.52,181.43,98,49.77,1.00,39.57,29.86,0.89,1522.02,2.34,4.266700e+02,4001.249,21.27
54,JVA,Consumer Defensive,Packaged Foods,216.36,262.41,203.68,170.25,122.10,98,3.14,0.88,2.55,1.77,0.68,838.09,0.20,4.750000e+00,13.333,15.70
56,CAVA,Consumer Cyclical,Restaurants,215.84,213.94,213.23,187.19,139.30,98,133.65,1.00,113.78,79.60,0.49,180.76,0.17,2.049000e+01,7.616,786.18
63,POWL,Industrials,Electrical Equipment & Parts,207.74,189.87,176.09,217.39,241.00,98,262.97,1.00,187.46,153.03,0.94,77.13,10.70,2.850000e+01,79.126,24.58
65,PI,Technology,Communication Equipment,206.20,199.35,179.78,180.55,134.98,98,235.65,1.00,181.43,144.41,0.66,158.71,0.28,1.449000e+01,11.532,841.61


SEZL, DAVE, CVNA, ADMA, APP, GGAL, JVA, CAVA, POWL, PI, BYRN, IESC, CHCI, USAP, SUPV, LWAY, PLTR, AGX, BBAR, VITL, SFM, PNTG, EAT, SN, CLMB, FUTU, NVDA, PTGX, VRT, HBB


In [95]:
# @title Filter 2. Increasing RS > 0 {"run":"auto"}
cond0 = "(df['RS'] > 100)" # @param ["(df['RS'] > 100)"]
cond1 = "" # @param ["& (df['RS'] > df['1wk:end max'])", ""]
cond2 = "" # @param ["& (df['1wk:end max'] > df['1mo:1wk max'])", ""]
cond3 = "& (df['1mo:1wk max'] > df['3mo:1mo max'])" # @param ["& (df['1mo:1wk max'] > df['3mo:1mo max'])", ""]
cond4 = "& (df['3mo:1mo max'] > df['6mo:3mo max'])" # @param ["& (df['3mo:1mo max'] > df['6mo:3mo max'])", ""]
cond5 = "& (df['6mo:3mo max'] > df['9mo:6mo max'])" # @param ["& (df['6mo:3mo max'] > df['9mo:6mo max'])", ""]
cond6 = "& (df['Rating (RS)'] > 90)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond7 = "& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond8 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond9 = "& (df['Rev RS'] > 0)" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond10 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond11 = "& (df['52W pos'] > 0.6)" # @param ["& (df['52W pos'] > 0.5)", "& (df['52W pos'] > 0.6)", "& (df['52W pos'] > 0.7)", ""]
cond12 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
num_items = 20 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 13
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df_top_f2 = df.head(num_items)
display(df_top_f2)

print_column(df_top_f2, 'Name')
print_column(df_top_f2, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 2y
RS window: 12mo
Date: 20241013


,Ticker,Sector,Industry,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Price,52W pos,MA50,MA200,Volume / VMA50,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE
37,ADMA,Healthcare,Biotechnology,244.17,303.96,289.22,232.09,147.39,98,17.15,0.80,17.60,10.23,1.60,347.68,0.14,71.42,1.447,122.46
54,JVA,Consumer Defensive,Packaged Foods,216.36,262.41,203.68,170.25,122.10,98,3.14,0.88,2.55,1.77,0.68,838.09,0.20,4.75,13.333,15.70
140,PNTG,Healthcare,Medical Care Facilities,165.04,192.74,184.71,155.27,138.04,96,33.79,0.88,32.93,23.58,0.68,67.84,0.64,13.78,20.380,52.80
199,PPIH,Industrials,Building Products & Equipment,149.92,149.51,123.54,102.18,90.34,95,13.44,0.89,11.00,9.06,0.60,186.67,1.90,4.01,19.878,7.07
231,CORT,Healthcare,Biotechnology,144.90,148.21,136.89,120.31,93.04,95,45.43,0.95,38.35,29.75,0.98,41.85,1.13,21.28,5.553,40.20
236,KARO,Technology,Software—Application,143.98,141.28,132.95,128.25,100.02,95,43.70,1.00,38.08,30.39,2.30,40.17,1.47,4.71,138.674,29.73
285,CALM,Consumer Defensive,Farm Products,137.97,129.82,123.01,107.94,106.35,94,90.45,1.00,73.54,62.85,1.03,5882.50,8.73,47.05,54.437,10.36
289,IAG,Basic Materials,Gold,137.59,173.29,161.24,159.59,129.43,94,4.83,0.80,4.94,3.75,1.00,125.90,0.24,29.13,2.519,20.12
291,RVLV,Consumer Cyclical,Internet Retail,137.17,141.98,126.32,125.11,121.83,94,25.35,0.96,23.37,19.44,0.66,73.44,0.46,1.84,14.902,55.11
294,GVA,Industrials,Engineering & Construction,136.76,138.36,133.81,128.27,114.77,94,81.78,1.00,74.82,60.74,0.90,460.79,2.01,6.49,86.530,40.69


ADMA, JVA, PNTG, PPIH, CORT, KARO, CALM, IAG, RVLV, GVA, TGLS, ATAT, PAYO, UG, JEF, MLI, SEI, LMNR


In [103]:
# @title Filter 3. RS Breakout {"run":"auto"}
base = 130 # @param [100, 105, 110, 115, 120, 130, 160, 200] {"type":"raw"}
cond0 = "(df['RS'] > base)" # @param ["(df['RS'] > base)"]
cond1 = "" # @param ["& (df['1wk:end max'] < base)",""]
cond2 = "" # @param ["& (df['1mo:1wk max'] < base)",""]
cond3 = "& (df['3mo:1mo max'] < base)" # @param ["& (df['3mo:1mo max'] < base)",""]
cond4 = "& (df['6mo:3mo max'] < base)" # @param ["& (df['6mo:3mo max'] < base)",""]
cond5 = "& (df['9mo:6mo max'] < base)" # @param ["& (df['9mo:6mo max'] < base)",""]
cond6 = "" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond7 = "& (df['Volume / VMA50'] > 2)" # @param ["& (df['Volume / VMA50'] > 1.5)", "& (df['Volume / VMA50'] > 2)", "& (df['Volume / VMA50'] > 3)", "& (df['Volume / VMA50'] > 4)", ""]
cond8 = "& (df['Price'] > df['MA200'])" # @param ["& (df['Price'] > df['MA50'])", "& (df['Price'] > df['MA200'])", ""]
cond9 = "& (df['MA50'] > df['MA200'])" # @param ["& (df['MA50'] > df['MA200'])",""]
cond10 = "& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond11 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond12 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond13 = "" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond14 = "& (df['52W pos'] > 0.7)" # @param ["& (df['52W pos'] > 0.5)", "& (df['52W pos'] > 0.6)", "& (df['52W pos'] > 0.7)", ""]
cond15 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", "& (df['Price'] < 500)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 16
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df_top_f3 = df.head(num_items)
display(df_top_f3)

print_column(df_top_f3, 'Name')
print_column(df_top_f3, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 2y
RS window: 12mo
Date: 20241013


,Ticker,Sector,Industry,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Price,52W pos,MA50,MA200,Volume / VMA50,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE
143,CIA,Financial Services,Insurance—Life,164.19,129.54,99.14,112.87,100.08,96,4.75,0.86,3.28,2.76,2.70,26.63,0.43,-4.38,4.960,11.05
271,NECB,Financial Services,Banks—Regional,139.20,137.37,127.99,104.71,105.52,94,27.30,1.00,23.00,18.48,3.04,50.19,3.59,8.17,7.827,7.60


CIA, NECB


In [97]:
# @title Filter 4. Groupby "Industry" {"run":"auto"}
cond0 = "(df['RS'] > 100)" # @param ["(df['RS'] > 100)"]
cond1 = "& (df['Rating (RS)'] > 95)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
num_items = 2 # @param [1, 2, 3, 4, 5] {"type":"raw"}

NUM_CONDS = 2

from vistock.ranking_utils import append_ratings, groupby_industry

print_metadata(metadata)
stock_df = df_merge.copy()

# Filter out rows with NaN in the 'Ticker' column
#stock_df = stock_df[stock_df['Ticker'].notna()]

rs_columns = ['RS',
              '1mo:1wk max', '3mo:1mo max', '6mo:3mo max', '9mo:6mo max']
if 'Name' in stock_df.columns:
    columns = ['Sector', 'Ticker', 'Name'] + rs_columns
else:
    columns = ['Sector', 'Ticker'] + rs_columns
df = groupby_industry(stock_df, columns, key='RS')

df = df.sort_values(by='RS', ascending=False).reset_index(drop=True)
rating_columns = ['Rating (RS)', 'Rating (1M:1W)', 'Rating (3M:1M)',
                  'Rating (6M:3M)', 'Rating (9M:6M)']
df = append_ratings(df, rs_columns, rating_columns)

cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))
df = df[eval(cond)]
df_top_f4 = df.head(num_items)
display(df_top_f4)

print_column(df_top_f4, 'Name')
print_column(df_top_f4, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 2y
RS window: 12mo
Date: 20241013


,Industry,Sector,Ticker,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Rating (1M:1W),Rating (3M:1M),Rating (6M:3M),Rating (9M:6M)
0,Utilities—Independent Power Producers,Utilities,"VST,TLN,NRG,PAM,TAC,KEN",141.92,154.61,140.11,150.03,130.41,99,99,98,98,88
1,Other Precious Metals & Mining,Basic Materials,"GATO,PPTA,EXK,HL,VOXR,MUX,TFPM,BVN,SBSW",119.11,132.34,137.53,148.68,122.01,98,98,98,97,82


VST,TLN,NRG,PAM,TAC,KEN, GATO,PPTA,EXK,HL,VOXR,MUX,TFPM,BVN,SBSW


In [98]:
# @title Filter 5. Financial {"run":"auto"}
cond0 = "((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond1 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond2 = "& (df['Rev RS'] > 0)" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond3 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond4 = "& (df['EPS YoY (%)'] > 0)" # @param ["& (df['EPS YoY (%)'] > 0)", "& ((df['EPS YoY (%)'] > 0) & (df['YoY 2Q Algo (%)'] > 0))", ""]
cond5 = "& ((df['EPS QoQ (%)'] > 0) & (df['QoQ 2Q Algo (%)'] > 0) & (df['QoQ 3Q Algo (%)'] > 0))" # @param ["& (df['EPS QoQ (%)'] > 0)", "& ((df['EPS QoQ (%)'] > 0) & (df['QoQ 2Q Algo (%)'] > 0))", "& ((df['EPS QoQ (%)'] > 0) & (df['QoQ 2Q Algo (%)'] > 0) & (df['QoQ 3Q Algo (%)'] > 0))", ""]
cond6 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
cond7 = "& (df['Sector'] == 'Technology')" # @param ["& (df['Sector'] == 'Technology')", "& (df['Sector'] == 'Energy')", ""]
sorted_column = "EPS RS" # @param ["EPS RS", "Rev RS"]
ascending = False # @param {"type":"boolean"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 8
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

df = df_fin.copy()

import numpy as np

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df[eval(cond)]
df = df.sort_values(by=sorted_column, ascending=ascending)
df_top_f5 = df.head(num_items)
display(df_top_f5)

print_column(df_top_f5, 'Name')
print_column(df_top_f5, 'Ticker')

,Ticker,Sector,Industry,Price,EPS QoQ (%),QoQ 2Q Algo (%),QoQ 3Q Algo (%),EPS YoY (%),YoY 2Q Algo (%),EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
14,NXT,Technology,Solar,33.960,0.12,0.14,0.05,0.95,37.25,22225695.51,3.78,34.22,27.133,9.43,99.0,87.0
269,VERX,Technology,Software—Application,39.650,0.50,2000000.00,2000000.00,2.67,7.00,903.97,0.13,6.77,4.021,310.38,93.0,71.0
416,PAY,Technology,Software—Infrastructure,21.150,0.33,0.09,0.10,0.60,5.00,506.89,0.26,21.75,5.646,87.62,90.0,83.0
487,PLTR,Technology,Software—Infrastructure,43.520,0.20,0.25,0.33,5.00,NaN,413.22,0.17,18.65,1.127,255.94,89.0,81.0
558,YOU,Technology,Software—Application,33.290,0.30,0.08,0.09,5.50,4.33,343.76,0.78,21.51,7.586,43.82,87.0,83.0
567,NVDA,Technology,Semiconductors,134.810,0.13,0.23,0.31,1.72,6.28,336.96,2.13,93.92,3.910,63.29,87.0,92.0
744,IMMR,Technology,Software—Application,8.320,0.52,0.22,5.12,3.14,NaN,242.74,1.80,875.30,5.119,4.72,84.0,95.0
772,APP,Technology,Software - Application,144.840,0.31,0.37,0.59,3.18,NaN,231.25,2.34,33.02,11.726,62.82,83.0,87.0
822,GB,Technology,Software—Infrastructure,5.250,0.25,0.33,23.32,1.50,NaN,213.54,0.13,48.43,1.894,40.54,82.0,89.0
1516,ODD,Technology,Software—Infrastructure,39.205,0.39,0.78,3.57,0.49,0.57,67.23,1.54,32.40,10.429,26.47,68.0,87.0


NXT, VERX, PAY, PLTR, YOU, NVDA, IMMR, APP, GB, ODD


## Step 3. Visualize Filtered Stocks

In [ ]:
# @title Plot 1. IBD Relative Strength Comparison {"run":"auto"}
source = "Filter 5. Financial" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout", "Filter 4. Groupby Industry", "Filter 5. Financial", "Major Global Stock Indices", "Sector Indices"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

MAX_STOCK_SELECTION = 10
N_COLS = 8  # the number of columns of the grid layout for checkboxes

symbols = {
    'Filter 1. Sorting': lambda: tickers_from_df(df_top_f1),
    'Filter 2. Increasing RS': lambda: tickers_from_df(df_top_f2),
    'Filter 3. RS Breakout': lambda: tickers_from_df(df_top_f3),
    'Filter 4. Groupby Industry': lambda: tickers_from_df(df_top_f4),
    'Filter 5. Financial': lambda: tickers_from_df(df_top_f5),
    'Major Global Stock Indices': major_indices,
    'Sector Indices': sector_indices,
}[source]()

checkboxes = cbs_create(symbols, MAX_STOCK_SELECTION)
checkbox_grid = cbs_with_grid(checkboxes, N_COLS)
btn_unselect_all = widgets.Button(description="Unselect All")
btn_unselect_all.on_click(lambda b: cbs_unselect_all(checkboxes))
btn_select_top = widgets.Button(description="Select Top 10")
btn_select_top.on_click(lambda b: cbs_select_top(checkboxes, MAX_STOCK_SELECTION))

period_dropdown = create_period_dropdown(metadata['Period'])
interval_dropown = create_interval_dropdown('1d')
rs_window_dropdown = create_rs_window_dropdown(metadata['RS window'])

cmp_theme_dropdown = {
    'mplfinance': create_style_dropdown('Comparison Theme:', 'charles'),
    'Plotly': create_template_dropdown('Comparison Theme:', 'plotly_dark'),
}[backend]
btn_plot = widgets.Button(description="Generate Plot")

out_msg, out_fig = widgets.Output(), widgets.Output()

ui = widgets.VBox([
    checkbox_grid,
    widgets.HBox([btn_unselect_all, btn_select_top]),
    widgets.VBox([period_dropdown, interval_dropown, rs_window_dropdown]),
    widgets.VBox([cmp_theme_dropdown, btn_plot]),
    out_msg, out_fig
])
display(ui)

import vistock.mpl as mpl
import vistock.plotly as ply

rs_cmp = {
    'mplfinance': mpl.ibd_rs_cmp,
    'Plotly': ply.ibd_rs_cmp,
}[backend]

def on_checkbox_change(change):
    selected_count = sum([cb.value for cb in checkboxes])
    if selected_count > MAX_STOCK_SELECTION:
        # Uncheck the last checked box if selection exceeds limit
        changed_checkbox = change['owner']
        changed_checkbox.value = False
        with out_msg:
            out_fig.clear_output()
            print(f"Only {MAX_STOCK_SELECTION} stocks can be selected at most.")

# Bind the checkbox change event to the function
for checkbox in checkboxes:
    checkbox.observe(on_checkbox_change, names='value')

def on_plot_click(b):
    symbols = cbs_get_selected(checkboxes)
    if not symbols:
        with out_msg:
            out_fig.clear_output()
            print("No stocks selected. Please select at least one stock.")
        return
    with out_fig:
        out_msg.clear_output()
        clear_output()
        interval = interval_dropown.value
        period = period_dropdown.value
        rs_window = rs_window_dropdown.value
        if rs_cmp is mpl.ibd_rs_cmp:
            rs_cmp.plot(symbols, interval=interval, period=period,
                        rs_window=rs_window,
                        style=cmp_theme_dropdown.value,
                        color_cycle=plt.cm.Paired.colors)
        else: # Plotly
            rs_cmp.plot(symbols, interval=interval, period=period,
                        rs_window=rs_window,
                        template=cmp_theme_dropdown.value,
                        colorway=px.colors.qualitative.Set3)

btn_plot.on_click(on_plot_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


In [ ]:
# @title Plot 2. IBD Stock Chart {"run":"auto"}
source = "Filter 5. Financial" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout","Filter 4. Groupby Industry","Filter 5. Financial","Major Global Stock Indices", "U.S. Listed Stocks", "Taiwan Stocks"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import functools as ft
import ipywidgets as widgets
from IPython.display import display, clear_output
import yfinance as yf

import vistock.stock_indices as si
import vistock.tw as tw

symbols = {
    'Filter 1. Sorting': lambda: tickers_from_df(df_top_f1),
    'Filter 2. Increasing RS': lambda: tickers_from_df(df_top_f2),
    'Filter 3. RS Breakout': lambda: tickers_from_df(df_top_f3),
    'Filter 4. Groupby Industry': lambda: tickers_from_df(df_top_f4),
    'Filter 5. Financial': lambda: tickers_from_df(df_top_f5),
    'Major Global Stock Indices': major_indices,
    'U.S. Listed Stocks': ft.partial(si.get_tickers, 'USLS'),
    'Taiwan Stocks': ft.partial(si.get_tickers, 'TWSE+TPEX+ESB'),
}[source]()
if source == 'Taiwan Stocks':
    symbols = [f"{tw_stock_name(s)} {s}" for s in symbols]

dropdowns, layout = create_search_dropdowns(symbols, 1)

period_dropdown2 = create_period_dropdown(metadata['Period'])
interval_dropown2 = create_interval_dropdown('1d')
rs_window_dropdown2 = create_rs_window_dropdown(metadata['RS window'])

stock_theme_dropdown = {
    'mplfinance': create_style_dropdown('Stock Theme:', 'yahoo'),
    'Plotly': create_template_dropdown('Stock Theme:', 'plotly'),
}[backend]

btn_plot_prc = widgets.Button(description="Price/RS/Volume Chart",
                              layout=widgets.Layout(width='168px'))
btn_plot_fin = widgets.Button(description="Financial Chart",
                              layout=widgets.Layout(width='168px'))
btn_report_q = widgets.Button(description="Quarterly Report",
                              layout=widgets.Layout(width='168px'))
btn_report_a = widgets.Button(description="Annual Report",
                              layout=widgets.Layout(width='168px'))
btn_clear_last = widgets.Button(description="Clear Last")
btn_clear_all = widgets.Button(description="Clear All")
#outputs = widgets.VBox()

ui = widgets.VBox([
    layout,
    period_dropdown2, interval_dropown2, rs_window_dropdown2, stock_theme_dropdown,
    widgets.HBox([btn_plot_prc, btn_plot_fin]),
    widgets.HBox([btn_report_q, btn_report_a]),
    widgets.HBox([btn_clear_last, btn_clear_all]),
    outputs
])
display(ui)

import vistock.mpl as mpl
import vistock.plotly as ply

stock_chart = {
    'mplfinance': mpl.ibd_rs,
    'Plotly': ply.ibd_rs,
}[backend]

def get_symbols():
    symbols = get_dropdowns_selected_options(dropdowns)
    if source == 'Taiwan Stocks':
        symbols = [s.split()[0] for s in symbols]
    return symbols

def on_plot_prc_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        interval = interval_dropown2.value
        period = period_dropdown2.value
        rs_window = rs_window_dropdown2.value
        if not selected:
            print("No Stock Selected!")
        elif stock_chart is mpl.ibd_rs:
            stock_chart.plot(selected[0], interval=interval, period=period,
                             rs_window=rs_window,
                             style=stock_theme_dropdown.value,
                             legend_loc='upper left')
        else: # Plotly
            stock_chart.plot(selected[0], interval=interval, period=period,
                             rs_window=rs_window,
                             template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

import vistock.mpl.financials as mpl_fin
import vistock.plotly.financials as ply_fin

fin_chart = {
    'mplfinance': mpl_fin,
    'Plotly': ply_fin,
}[backend]

def on_plot_fin_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        elif fin_chart is mpl_fin:
            fin_chart.plot(selected[0], style=stock_theme_dropdown.value)
        else: # Plotly
            fin_chart.plot(selected[0], template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

def on_report_q_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Quarterly Financials:")
            display(yf.Ticker(ticker).quarterly_financials)
    outputs.children = (new_output,) + outputs.children

def on_report_a_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Annual Financials:")
            display(yf.Ticker(ticker).financials)
    outputs.children = (new_output,) + outputs.children

def on_clear_last_click(b):
    if outputs.children:
        children = list(outputs.children)
        children.pop(0)
        outputs.children = tuple(children)

def on_clear_all_click(b):
    outputs.children = ()

btn_plot_prc.on_click(on_plot_prc_click)
btn_plot_fin.on_click(on_plot_fin_click)
btn_report_q.on_click(on_report_q_click)
btn_report_a.on_click(on_report_a_click)
btn_clear_last.on_click(on_clear_last_click)
btn_clear_all.on_click(on_clear_all_click)

if backend == 'Plotly':
    enable_plotly_in_cell()
